In [1]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\Working_attributes")
from Imports import * 

In [54]:
print_to_excel = True

In [2]:
# сбор данных из ручного источника
file_name = 'Brazil_accounts_full_cover.xlsx'
data_excel = pd.ExcelFile(file_name)
sheet_list = data_excel.sheet_names
data = pd.DataFrame()
for sheet in sheet_list:
    data_comp = pd.read_excel(file_name, sheet_name=sheet)
    if sheet=='BRFTO':
        data_comp.columns = ['glAccount','dis','code_4','bscCode','bcsAccount']
    else:
        data_comp.columns = ['glAccount','dis','bscCode','bcsAccount']
    data_comp = data_comp[~data_comp.bcsAccount.isna()]
    data_comp['buCode'] = sheet
    data = pd.concat([data, data_comp], axis=0)
data.shape

(178, 6)

In [21]:
data_glAccount_list = data.glAccount.dropna().unique().tolist() # mb glAccount
data_bscCode_list = data.bscCode.dropna().unique().tolist() # bscCode
data_bcsAccount_list = data.bcsAccount.dropna().unique().tolist() # bcsAccount
data_code_4_list = data.code_4.dropna().unique().tolist() # glAccount

In [17]:
# # выгружаем полные данные для проверки вхождени столбцов
# data_ARAP_full = export_from_RISKCUSTOM(f'''
# SELECT distinct *
# from RISKACCESS."unifiedARReport" 
# where "reportDate" = TO_DATE('2024-08-31', 'YYYY-MM-DD')
# --and "companyCode" in ({str(sheet_list)[1:-1]})
# fetch first 50000 rows only 
# ''')

# # проверка вхождения столбцов
# s = (data_ARAP_full.isin(data_glAccount_list)).any(axis=0)
# s[s==True]

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


(50000, 166)

In [28]:
query = f'''
SELECT "companyCode","glAccount","bscCode","bcsAccount","currencyCode","presentAmountUSD"
            from RISKACCESS."unifiedARReport" 
            where "reportDate" = TO_DATE('2024-08-31', 'YYYY-MM-DD')
AND "companyCode" in ({str(sheet_list)[1:-1]})
AND (
    "bscCode" in ({str(data_bscCode_list)[1:-1]}) 
    OR "bcsAccount" in ({str(data_bcsAccount_list)[1:-1]})
    OR "glAccount" in ({str(data_glAccount_list)[1:-1]})  
    OR "glAccount" in ({str(data_code_4_list)[1:-1]})  
    )
'''
data_ARAP = export_from_RISKCUSTOM(query)
data_ARAP.shape

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


(1131, 6)

In [26]:
data_ARAP.companyCode.unique()

array(['BRFTO', 'BRHRG'], dtype=object)

In [72]:
data_ARAP_cols_list = ['companyCode','glAccount','bscCode','bcsAccount','currencyCode']
data_ARAP_pivot = data_ARAP.groupby(as_index=False, dropna=False, by=data_ARAP_cols_list)['presentAmountUSD'].agg('sum')

data_cols_list = ['buCode','glAccount','code_4','bscCode','bcsAccount']
data_for_merge = data[data_cols_list].drop_duplicates().reset_index(drop=True)

# merge 
merge_dict = {'glAccount':'glAccount', 'bscCode':'bscCode', 'bcsAccount':'bcsAccount'}
# , 'code_4':'glAccount'
# bucode + glAccount on glAccount
data_for_merge.glAccount, data_ARAP_pivot.glAccount = data_for_merge.glAccount.astype(int), data_ARAP_pivot.glAccount.astype(int)
# bucode + glAccount on code_4
data_for_merge.code_4 = data_for_merge.code_4.fillna(0).astype(int)
# bucode + bscCode
data_for_merge.bscCode, data_ARAP_pivot.bscCode = data_for_merge.bscCode.astype(str).replace(' ',''), data_ARAP_pivot.bscCode.astype(str).replace(' ','')
# bucode + bcsAccount
data_for_merge.bcsAccount, data_ARAP_pivot.bcsAccount = data_for_merge.bcsAccount.astype(str).replace(' ',''), data_ARAP_pivot.bcsAccount.astype(str).replace(' ','')

full_data_merge = pd.DataFrame()
for key in list(merge_dict.keys()):
    left_on = ['buCode'] + [key]
    right_on = ['companyCode'] + [key]
    data_merge = data_for_merge.merge(data_ARAP_pivot, how='inner', left_on=left_on, right_on=right_on)
    data_merge['merge_on'] = key
    if len(data_merge) != 0:
        full_data_merge = pd.concat([full_data_merge,data_merge])
full_data_merge

,buCode,glAccount,code_4,bscCode_x,bcsAccount_x,companyCode,bscCode_y,bcsAccount_y,currencyCode,presentAmountUSD,merge_on,glAccount_x,bscCode,glAccount_y
0,BRHRG,4.500000e+05,0,BSC350,B661000,BRHRG,nan,B769910,BRL,-7.029793e+06,glAccount,NaN,NaN,NaN
1,BRHRG,4.602100e+05,0,BSC350,B661000,BRHRG,nan,B769910,BRL,0.000000e+00,glAccount,NaN,NaN,NaN
2,BRHRG,2.150180e+05,0,BSC350,B661000,BRHRG,nan,B769910,BRL,1.136868e-13,glAccount,NaN,NaN,NaN
3,BRHRG,2.150280e+05,0,BSC350,B661000,BRHRG,nan,B769910,BRL,-2.287862e+06,glAccount,NaN,NaN,NaN
4,BRHRG,6.220100e+05,0,BSC400,B673100,BRHRG,nan,B769910,BRL,-9.313226e-10,glAccount,NaN,NaN,NaN
5,BRHRG,6.220120e+05,0,BSC400,B673100,BRHRG,nan,B769910,BRL,0.000000e+00,glAccount,NaN,NaN,NaN
6,BRHRG,6.300020e+05,0,BSC400,B673100,BRHRG,nan,B769910,BRL,-1.721574e+07,glAccount,NaN,NaN,NaN
7,BRHRG,6.310020e+05,0,BSC400,B673100,BRHRG,nan,B769910,BRL,-3.931064e+06,glAccount,NaN,NaN,NaN
8,BRHRG,6.310020e+05,0,BSC400,B673100,BRHRG,nan,B769910,USD,-7.317683e+07,glAccount,NaN,NaN,NaN
9,BRHRG,6.200030e+05,0,BSC400,B673100,BRHRG,nan,B769910,BRL,0.000000e+00,glAccount,NaN,NaN,NaN


In [69]:
# data_for_merge.merge(data_ARAP_pivot, how='inner', left_on=['buCode', 'code_4'], right_on=['companyCode', 'glAccount'])
# set(data_for_merge.glAccount.unique()).intersection(set(data_ARAP_pivot.glAccount.unique()))

,buCode,glAccount_x,code_4,bscCode_x,bcsAccount_x,companyCode,glAccount_y,bscCode_y,bcsAccount_y,currencyCode,presentAmountUSD


In [73]:
if print_to_excel == True:
    Output_file = f'{str(date.today())}_BRL_ARAP_full_cover.xlsx'
    with pd.ExcelWriter(Output_file, engine='openpyxl') as writer:
        pd.DataFrame({'SQL':query}, index=[1]).to_excel(writer, index=False, sheet_name='SQL')
        data.to_excel(writer, index=False, sheet_name='full_cover')
        data_ARAP.to_excel(writer, index=False, sheet_name='ARAP')
        full_data_merge.to_excel(writer, index=False, sheet_name='merge')